In [1]:
# 필요한 패키지 설치
! pip install -Uq langchain_openai langchain langchain-community streamlit pypdf faiss-cpu


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
! pip install -qU langchain-teddynote


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
! pip install --upgrade langchain


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
! pip install langchain_experimental


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
! pip install tabulate


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
! pip install pandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
! pip install streamlit


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%%writefile chatbot.py

import os
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader, DataFrameLoader
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain.prompts import ChatPromptTemplate
import tempfile
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor
import pandas as pd

# .env 파일 로드
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

########## 1. 폴더 내 파일 로드 ##########

# 폴더 경로 설정
folder_path = "./data"  # 분석할 파일이 저장된 폴더 경로
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)

# PDF 문서 로드 함수
def load_pdf_with_metadata(file_path):
    loader = PyMuPDFLoader(file_path)
    documents = loader.load_and_split(text_splitter)
    for doc in documents:
        doc.metadata["source"] = os.path.basename(file_path)
        doc.metadata["page"] = doc.metadata.get("page", "Unknown")
    return documents

# 엑셀 문서 로드 함수
def load_excel_with_metadata(file_path):
    documents = []
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        loader = DataFrameLoader(df, page_content_column=df.columns[0])
        sheet_docs = loader.load_and_split(text_splitter)
        for doc in sheet_docs:
            doc.metadata["source"] = os.path.basename(file_path)
            doc.metadata["sheet_name"] = sheet_name
            doc.metadata["cell_range"] = f"A1:{df.columns[-1]}{len(df)}"  # 추가 셀 범위 정보
        documents.extend(sheet_docs)
    return documents


# CSV 문서 로드 함수
def load_csv_with_metadata(file_path):
    documents = []
    df = pd.read_csv(file_path)
    loader = DataFrameLoader(df, page_content_column=df.columns[0])
    csv_docs = loader.load_and_split(text_splitter)
    for doc in csv_docs:
        doc.metadata["source"] = os.path.basename(file_path)
        doc.metadata["cell_range"] = f"A1:{df.columns[-1]}{len(df)}"  # 추가 셀 범위 정보
    documents.extend(csv_docs)
    return documents

# 폴더 내 모든 문서를 로드

def load_documents_from_folder(folder_path):
    documents = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.endswith(".pdf"):
            documents.extend(load_pdf_with_metadata(file_path))
        elif file_name.endswith(".xlsx") or file_name.endswith(".xls"):
            documents.extend(load_excel_with_metadata(file_path))
        elif file_name.endswith(".csv"):
            documents.extend(load_csv_with_metadata(file_path))
    return documents



# 에이전트와 대화하는 함수
def chat_with_agent(user_input, agent_executor):
    result = agent_executor({"input": user_input})
    response = result['output']  # 명시적으로 출력 키를 처리
    return response

# 세션 기록 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in st.session_state.session_history:
        st.session_state.session_history[session_ids] = ChatMessageHistory()
    return st.session_state.session_history[session_ids]

# 대화 내용 출력하는 함수
def print_messages():
    for msg in st.session_state["messages"]:
        st.chat_message(msg['role']).write(msg['content'])


# 모든 문서 로드
all_docs = load_documents_from_folder(folder_path)


# FAISS 인덱스 설정 및 생성
vector = FAISS.from_documents(all_docs, OpenAIEmbeddings())
retriever = vector.as_retriever()

# 도구 정의
retriever_tool = create_retriever_tool(
    retriever,
    name="csv_search",
    description="Use this tool to search information from the csv document"
)

# Streamlit 메인 코드
def main():
    # 페이지 설정
    st.set_page_config(page_title="AI 비서", layout="wide", page_icon="🤖")

    st.image('Marporter.png', width=1200)
    st.markdown('---')
    st.title("안녕하세요! RAG를 활용한 'AI 비서 대동이' 입니다")  # 시작 타이틀

    # 세션 초기화
    if "messages" not in st.session_state:
        st.session_state["messages"] = []

    if "session_history" not in st.session_state:
        st.session_state["session_history"] = {}

    with st.sidebar:
        st.session_state["OPENAI_API"] = st.text_input(label="OPENAI API 키", placeholder="Enter Your API Key", value="", type="password")
        st.markdown('---')

    # OpenAI API 키가 입력되었는지 확인
    if st.session_state["OPENAI_API"] :
        os.environ['OPENAI_API_KEY'] = st.session_state["OPENAI_API"]

        # return retriever_tool
        tools = [retriever_tool]

        # LLM 설정
        llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

        # Prompt 정의
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You Should answer the user's questions in a friendly and kind manner. And should answer in Korean.
                    You are a 15-year veteran market information analyst specializing in agriculture, agricultural machinery, future agriculture, and smart mobility in our company.
                    Our company is a farming machinery firm called '대동'. And Representative domestic competitors are 'TYM' and 'LS엠트론'.
                    Our company revenue structure is based on two main channels: domestic sales in South Korea and exports to regions such as Southeast Asia, the United States, and Europe. The revenue comes from parts, products, and services.
                    Analyze the 'Title', 'Subheading', 'Content', and 'date' columns of the dataframe (df) to classify each article into one of the following categories: [정치/사회], [경쟁사 정보], [시장 정보] and [기술 동향].
                    You have news articles from the last two weeks related to keywords such as [정치/사회], [경쟁사 정보], [시장 정보] and [기술 동향].
                    When the chat begins, you introduce yourself and ask the user for a keywords in [정치/사회], [경쟁사 정보], [시장 정보] and [기술 동향] to search.
                    If the keyword provided by the user does not match the pre-defined keyword format, you determine the user's intent and confirm if it matches a request related to the keywords [정치/사회], [경쟁사 정보], [시장 정보] or [기술 동향].
                    Please answer questions following the format [FORMAT] below.
                    
                    '기사 제목' is 

                    #FORMAT
                    <기사 제목>
                    일자 :  

                    * 요약
                    -
                    -
                    -

                    출처 :              
                        """

                ),
                ("placeholder", "{chat_history}"),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]
        )

        # 에이전트 생성 (initialize_agent 대신 create_tool_calling_agent 사용)
        agent = create_tool_calling_agent(llm, tools, prompt)

        # AgentExecutor 정의
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

        # 사용자 입력 처리
        user_input = st.chat_input('질문이 무엇인가요?')

        if user_input:
            session_id = "default_session"
            session_history = get_session_history(session_id)

            if session_history.messages:
                previous_messages = [{"role": msg['role'], "content": msg['content']} for msg in session_history.messages]
                response = chat_with_agent(user_input + "\n\nPrevious Messages: " + str(previous_messages), agent_executor)
            else:
                response = chat_with_agent(user_input, agent_executor)

            # 메시지를 세션에 추가
            st.session_state["messages"].append({"role": "user", "content": user_input})
            st.session_state["messages"].append({"role": "assistant", "content": response})

            # 세션 기록에 메시지를 추가
            session_history.add_message({"role": "user", "content": user_input})
            session_history.add_message({"role": "assistant", "content": response})

        # 대화 내용 출력
        print_messages()

    else:
        st.warning("OpenAI API 키를 입력하세요")

if __name__ == "__main__":
    main()

Overwriting chatbot.py


In [ ]:
%%writefile chatbot.py

import os
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader, DataFrameLoader
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.tools.retriever import create_retriever_tool
from langchain.prompts import ChatPromptTemplate
import tempfile
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor
import pandas as pd
# .env 파일 로드
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
########## 1. 폴더 내 파일 로드 ##########

# 폴더 경로 설정
folder_path = "./data"  # 분석할 파일이 저장된 폴더 경로
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)

# PDF 문서 로드 함수
def load_pdf_with_metadata(file_path):
    loader = PyMuPDFLoader(file_path)
    documents = loader.load_and_split(text_splitter)
    for doc in documents:
        doc.metadata["source"] = os.path.basename(file_path)
        doc.metadata["page"] = doc.metadata.get("page", "Unknown")
    return documents

# 엑셀 문서 로드 함수
def load_excel_with_metadata(file_path):
    documents = []
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        loader = DataFrameLoader(df, page_content_column=df.columns[0])
        sheet_docs = loader.load_and_split(text_splitter)
        for doc in sheet_docs:
            doc.metadata["source"] = os.path.basename(file_path)
            doc.metadata["sheet_name"] = sheet_name
            doc.metadata["cell_range"] = f"A1:{df.columns[-1]}{len(df)}"  # 추가 셀 범위 정보
        documents.extend(sheet_docs)
    return documents


# CSV 문서 로드 함수
def load_csv_with_metadata(file_path):
    documents = []
    df = pd.read_csv(file_path)
    loader = DataFrameLoader(df, page_content_column=df.columns[0])
    csv_docs = loader.load_and_split(text_splitter)
    for doc in csv_docs:
        doc.metadata["source"] = os.path.basename(file_path)
        doc.metadata["cell_range"] = f"A1:{df.columns[-1]}{len(df)}"  # 추가 셀 범위 정보
    documents.extend(csv_docs)
    return documents

# 폴더 내 모든 문서를 로드

def load_documents_from_folder(folder_path):
    documents = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.endswith(".pdf"):
            documents.extend(load_pdf_with_metadata(file_path))
        elif file_name.endswith(".xlsx") or file_name.endswith(".xls"):
            documents.extend(load_excel_with_metadata(file_path))
        elif file_name.endswith(".csv"):
            documents.extend(load_csv_with_metadata(file_path))
    return documents



# 에이전트와 대화하는 함수
def chat_with_agent(user_input, agent_executor):
    result = agent_executor({"input": user_input})
    response = result['output']  # 명시적으로 출력 키를 처리
    return response

# 세션 기록 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in st.session_state.session_history:
        st.session_state.session_history[session_ids] = ChatMessageHistory()
    return st.session_state.session_history[session_ids]

# 대화 내용 출력하는 함수
def print_messages():
    for msg in st.session_state["messages"]:
        st.chat_message(msg['role']).write(msg['content'])


# 모든 문서 로드
all_docs = load_documents_from_folder(folder_path)


# FAISS 인덱스 설정 및 생성
vector = FAISS.from_documents(all_docs, OpenAIEmbeddings())
retriever = vector.as_retriever()

# 도구 정의
retriever_tool = create_retriever_tool(
    retriever,
    name="csv_search",
    description="Use this tool to search information from the csv document"
)


# Streamlit 메인 코드
def main():
    # 페이지 설정
    st.set_page_config(page_title="마포터", layout="wide", page_icon="🤖")

    st.image('chatbot_image.png', width=600)
    st.markdown('---')
    st.title("안녕하세요! 대동 마포터 입니다")  # 시작 타이틀

    # 세션 초기화
    if "messages" not in st.session_state:
        st.session_state["messages"] = []

    if "session_history" not in st.session_state:
        st.session_state["session_history"] = {}

    # with st.sidebar:
    #     st.session_state["OPENAI_API"] = st.text_input(label="OPENAI API 키", placeholder="Enter Your API Key", value="", type="password")
    #     st.markdown('---')

    # # OpenAI API 키가 입력되었는지 확인
    # if st.session_state["OPENAI_API"] :
    #     os.environ['OPENAI_API_KEY'] = st.session_state["OPENAI_API"]

# return retriever_tool
    tools = [retriever_tool]

    # LLM 설정
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

    # Prompt 정의
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                You Should answer the user's questions in a friendly and kind manner. And should answer in Korean.
                Your name is `마포터`.
                You are a 15-year veteran market information analyst specializing in agriculture, agricultural machinery, future agriculture, and smart mobility in our company.
                Our company is a farming machinery firm called '대동'. and Representative domestic competitors include 'TYM' and 'LS엠트론'.
                Our company revenue structure is based on two main channels: domestic sales in South Korea and exports to regions such as Southeast Asia, the United States, and Europe. The revenue comes from parts, products, and services.
                Analyze the 'Title', 'Subheading', 'Content', and 'date' columns of the dataframe (df) to classify each article into one of the following categories: [정치/사회], [경쟁사 정보], [시장 정보] and [기술 동향].
                You have news articles from the last two weeks related to keywords such as [정치/사회], [경쟁사 정보], [시장 정보] and [기술 동향].
                When the chatbot begins, you should introduce yourself and ask the user for a keywords to search.
                If the keyword provided by the user does not match the pre-defined keyword format, you determine the user's intent and confirm if it matches a request related to the keywords [정치/사회], [경쟁사 정보], [시장 정보] or [기술 동향].
                Please answer questions following the format [FORMAT] below.
                `요약` should include a condensed version of the article's content. 
                Include the title in the `기사 제목` section.   
                    

                #FORMAT
                [기사 제목] 
                일자 :  

                * 요약
                -
                -
                -

                출처 :              
                    
                    """

            ),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ]
    )

    # 에이전트 생성 (initialize_agent 대신 create_tool_calling_agent 사용)
    agent = create_tool_calling_agent(llm, tools, prompt)

    # AgentExecutor 정의
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    # 사용자 입력 처리
    user_input = st.chat_input('질문이 무엇인가요?')

    if user_input:
        session_id = "default_session"
        session_history = get_session_history(session_id)
        
        if session_history.messages:
            previous_messages = [{"role": msg['role'], "content": msg['content']} for msg in session_history.messages]
            response = chat_with_agent(user_input + "\n\nPrevious Messages: " + str(previous_messages), agent_executor)
        else:
            response = chat_with_agent(user_input, agent_executor)

    
    

        # 메시지를 세션에 추가
        st.session_state["messages"].append({"role": "user", "content": user_input})
        st.session_state["messages"].append({"role": "assistant", "content": response})

        # 세션 기록에 메시지를 추가
        session_history.add_message({"role": "user", "content": user_input})
        session_history.add_message({"role": "assistant", "content": response})

    # 대화 내용 출력
    print_messages()

    # else:
    #     st.warning("OpenAI API 키를 입력하세요")

if __name__ == "__main__":
    main()


Overwriting chatbot.py


In [ ]:
%%writefile chatbot.py

import os
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.tools.retriever import create_retriever_tool
from langchain.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from dotenv import load_dotenv
import pandas as pd
from langchain.docstore.document import Document
from langchain.tools import Tool  # 추가 임포트


# .env 파일 로드
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ['OPENAI_API_KEY'] = st.secrets["OPENAI_API_KEY"]
# .env 파일 로드

def load_csv_data(csv_path):
    # CSV 파일에서 기사 데이터를 로드
    df = pd.read_csv(csv_path)
    
    # 결측값 처리 (NaN 값을 빈 문자열로 대체)
    df = df.fillna("")
    
    # 기사 데이터를 Document 형태로 변환
    documents = [
        Document(
            page_content=row['Content'],
            metadata={"title": row['Title'], "date": row['date'], "url": row['URL']}
        )
        for _, row in df.iterrows()
        if row['Content'].strip()  # Content가 비어 있지 않은 경우만 처리
    ]
    
    # 텍스트 분할기 설정
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
    split_docs = text_splitter.split_documents(documents)

    # FAISS 인덱스 생성
    vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())
    retriever = vector.as_retriever()

    # 도구로 변환
    tool = Tool(
        name="news_search",
        func=retriever.get_relevant_documents,
        description="Search for relevant agricultural news articles.",
    )

    return tool

def load_pdf_files(pdf_paths):
    all_documents = []

    for pdf_path in pdf_paths:
        # PyPDFLoader를 사용하여 파일 로드
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        all_documents.extend(documents)

    # 텍스트 분할기 설정
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
    split_docs = text_splitter.split_documents(all_documents)

    # FAISS 인덱스 설정 및 생성
    vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())
    retriever = vector.as_retriever()

    # 도구 정의
    retriever_tool = create_retriever_tool(
        retriever,
        name="pdf_search",
        description="Use this tool to search information from the PDF document."
    )
    return retriever_tool

# 에이전트와 대화하는 함수
def chat_with_agent(user_input, agent_executor):
    result = agent_executor({"input": user_input})
    response = result['output']  # 명시적으로 출력 키를 처리
    return response

# 대화 내용 출력하는 함수
def print_messages():
    for msg in st.session_state["messages"]:
        st.chat_message(msg['role']).write(msg['content'])

# Streamlit 메인 코드
def main():
    # 페이지 설정
    st.set_page_config(page_title="농업 뉴스 Q&A", layout="wide", page_icon="🌾")

    st.image('Maporter_image.png', width=600)
    st.markdown('---')
    st.title("안녕하세요! '대동 마포터' 입니다")  # 시작 타이틀

    # 세션 초기화
    if "messages" not in st.session_state:
        st.session_state["messages"] = []

    # 특정 PDF 경로 지정
    # CSV 데이터 로드
    csv_path = './data/news_classified_updated_utf8.csv'
    if csv_path:
        pdf_search = load_csv_data(csv_path)
        tools = [pdf_search]

        # LLM 설정
        llm = ChatOpenAI(model_name="gpt-4", temperature=0)

        # 프롬프트 설정
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system",
                 "Be sure to answer in Korean. You are a helpful assistant. "
                 "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
                 "Please always include emojis in your responses with a friendly tone. "
                 "Your name is `대동 마포터`. Please introduce yourself at the beginning of the conversation."
                 ''' 
                 You are a very friendly chatbot called `대동 마포터`. \n\n
                 At the beginning of the chatbot conversation, always display the message: "I am a friendly assistant providing various insights related to agriculture **`based on the latest collected news.`**"\n\n
                 "You are a helpful assistant providing insights based on agricultural news articles collected. "
                 "Always respond in a professional and friendly tone with structured and informative answers."),
                Please enter the area you're curious about in relation to agriculture: `농업과 관련한 [정치/사회], [경쟁사 정보], [시장 정보] [기술 동향] 분야 중 궁금한 분야를 입력해주세요. `
                 [경쟁사 정보]: Our company `대동`, develops tractors, and our competitors include `LS엠트론`, `TYM`, and `존 디어`.
                [시장 정보]: For articles containing general information about the agricultural market.
                [기술 동향]: For articles related to the latest or new technologies in the agricultural field, or articles focused on technology.
                 
                 When summarizing a news article in your response, always use the following #format:
                For topics related to [정치/사회], [경쟁사 정보], [시장 정보] [기술 동향] in agriculture, please find and summarize five relevant news articles using the format above.
                #Format

                **[기사 제목]** Title of the news article \n\n
                **[기사 날짜]** Date of the news article \n\n
                **[뉴스 내용]** First, provide a 2–3 sentence summary of the overall news content. Then, summarize the main points again in a bullet point format. \n\n
                **[뉴스 출처]** URL link to the original news article
                 
                 '''),
                ("placeholder", "{chat_history}"),
                ("human", "{input} \n\n Be sure to include emoji in your responses."),
                ("placeholder", "{agent_scratchpad}"),
            ]
        )

        # 에이전트 생성
        agent = create_tool_calling_agent(llm, tools, prompt)

        # AgentExecutor 정의
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

        # 사용자 입력 처리
        user_input = st.chat_input('질문이 무엇인가요?')

        if user_input:
            response = chat_with_agent(user_input, agent_executor)

            # 메시지를 세션에 추가
            st.session_state["messages"].append({"role": "user", "content": user_input})
            st.session_state["messages"].append({"role": "assistant", "content": response})

        # 대화 내용 출력
        print_messages()

if __name__ == "__main__":
    main()


Overwriting chatbot1.py


In [ ]:
pip install --upgrade langchain
